# Python for Data Science - Final Project - Team 4
## Performance Analysis of Fast Food Companies in the Stock Market

### Getting Started
This notebook is designed to provide a structured approach to exploring and analyzing the datasets of 10 major fast-food companies.
Each dataset contains daily stock market data with information on opening price, closing price, trading volume, and other financial metrics.

### Objective
The primary goal of this notebook is to perform a comprehensive analysis of the stock market performance of these companies over the years. Through this analysis, we aim to:

1. Understand the structure and content of the datasets
2. Explore key metrics such as price trends, trading volumees, and price spreads
3. Visualize and interrupt important patterns and relationships in the data
4. Summarize insights that reveal the financial dynamics of the fast-food industry

### Dataset Overview
The datasets include the following companies:

- BRK-A: Berkshire Hathaway Inc.
- DNUT: Krispy Kreme, Inc.
- DPZ: Domino’s Pizza, Inc.
- LKNCY: Luckin Coffee Inc.
- MCD: McDonald’s Corporation
- PZZA: Papa John’s International, Inc.
- QSR: Restaurant Brands International, Inc.
- SBUX: Starbucks Corporation
- WEN: The Wendy’s Corporation
- YUM: Yum! Brands, Inc.

Each dataset contains the following columns:

- *Date*: Trading date.
- *Open*: Opening price.
- *High*: Highest price during the session.
- *Low*: Lowest price during the session.
- *Close*: Closing price.
- *Adj Close*: Adjusted closing price (accounting for dividends and stock splits).
- *Volume*: Number of shares traded during the session.

---

# Data Preprocessing Summary

## Preprocessing Steps

1. **Missing Volume Handling**:
   - Replaced missing values in the `Volume` column using a 7-day rolling mean.

2. **Missing Values Imputation**:
   - For columns `['Open', 'High', 'Low', 'Close', 'Adj Close']`:
     - Replaced outliers (Z-score > 3 or < -3) with the rolling median (7-day window).
     - Imputed any remaining missing values using the rolling median.

3. **Erroneous Values Correction**:
   - Checked for rows where `Low >= High`:
     - Replaced `Low` values with a 7-day rolling mean.
     - Ensured that `Low` values do not exceed corresponding `High` values by adjusting `Low` to equal `High` where necessary.

4. **Zero Volume Handling**:
   - For rows where `Volume == 0`:
     - If price columns (`['Open', 'High', 'Low', 'Close']`) had different values, replaced them with the mode (most common value).

5. **Uniform Price Handling**:
   - Identified rows where all price columns (`['Open', 'High', 'Low', 'Close']`) were identical:
     - If `Volume` was NaN, replaced it with `0.0`.

6. **Duplicate Dates Check**:
   - Checked for duplicate dates in the `Date` column:
     - Reported the count of duplicates and displayed duplicate rows if found.
     - If no duplicates, confirmed the absence of duplicate dates.

7. **Output**:
   - Saved the cleaned DataFrame for each ticker into a dictionary (`dfs`) for further analysis.

## Key Points
- Used rolling statistics (mean and median) for robust handling of missing values and outliers.
- Ensured consistency in `Low` and `High` values.
- Imputed `Volume` as `0.0` for rows with identical price columns and missing volume.


In [18]:
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
import numpy as np

# Adding the files into a list
files=['/Users/marioscontopoulos/Downloads/ProjectDatasets/Team_4/BRK-A.csv',
       '/Users/marioscontopoulos/Downloads/ProjectDatasets/Team_4/DNUT.csv',
       '/Users/marioscontopoulos/Downloads/ProjectDatasets/Team_4/DPZ.csv',
       '/Users/marioscontopoulos/Downloads/ProjectDatasets/Team_4/LKNCY.csv',
       '/Users/marioscontopoulos/Downloads/ProjectDatasets/Team_4/MCD.csv',
       '/Users/marioscontopoulos/Downloads/ProjectDatasets/Team_4/PZZA.csv',
       '/Users/marioscontopoulos/Downloads/ProjectDatasets/Team_4/QSR.csv',
       '/Users/marioscontopoulos/Downloads/ProjectDatasets/Team_4/SBUX.csv',
       '/Users/marioscontopoulos/Downloads/ProjectDatasets/Team_4/WEN.csv',
       '/Users/marioscontopoulos/Downloads/ProjectDatasets/Team_4/YUM.csv']

i=0

# Step 1: Create a list to store the DataFrames
dfs = {}
tickers = ['BRK-A', 'DNUT', 'DPZ', 'LKNCY', 'MCD', 'PZZA', 'QSR', 'SBUX', 'WEN', 'YUM']

for ticker, file in zip(tickers, files):

    # Step 1: Load the dataset
    df = pd.read_csv(file, index_col=False)

    print(df.isnull().sum())

    print('initialdataset',df.head(16))

    # Handling Missing Volume
    df['Volume'] =df['Volume'].fillna(df['Volume'].rolling(window=7, min_periods=1).mean())

    
    # Impute missing values for each column using rolling mean
    columns_to_impute = ['Open', 'High', 'Low', 'Close', 'Adj Close']
    # Handling Outliers
    for column in columns_to_impute:
        # Calculate mean and standard deviation
        mean = df[column].mean()
        std_dev = df[column].std()

        # Calculate Z-scores
        z_scores = (df[column] - mean) / std_dev

        # Identify outlier indices (Z-score > 3 or < -3)
        outlier_indices = df[np.abs(z_scores) > 3].index

        # Replace outliers with the rolling median
        for idx in outlier_indices:
            df.at[idx, column] = df[column].rolling(window=7, min_periods=1).median().iloc[idx]

        # Fill any remaining missing values with rolling median
        df[column] = df[column].fillna(df[column].rolling(window=7, min_periods=1).median())

   

    
    # Step 1: Check for rows where Low >= High
    erroneous_rows = df['Low'] >= df['High']

    # Step 2: Replace erroneous values with the column's average
    low_avg = df['Low'].rolling(window=7, min_periods=1).mean()
    
    df.loc[erroneous_rows, 'Low'] = low_avg
    #INCLUDE IF AVERAGE OF LOW IS HIGHER THAN HIGH VALUE, THEN REPLCAE THE LOW WITH THE HIGH VALUE SO THEY ARE THE SAME

    # Step 1: Identify rows where Volume == 0
    volume_zero_rows = df['Volume'] == 0

    # Step 2: Apply the logic to each row where Volume is 0
    for index, row in df[volume_zero_rows].iterrows():
        # Step 3: Check if any of Open, High, Low, or Close are different
        values = ['Open', 'High', 'Low', 'Close']
        values_dict = row[values]

        # If any value is different, replace it with the common value
        unique_values = values_dict.unique()
        if len(unique_values) > 1:  # More than one unique value means there is a difference
            common_value = values_dict.mode()[0]  # Find the most common value
            for col in values:
                if row[col] != common_value:
                    df.at[index, col] = common_value  # Replace with the common value


    # Step 1: Identify rows where all price columns are the same
    price_columns = ['Open', 'High', 'Low', 'Close']
    price_same_rows = (df[price_columns].nunique(axis=1) == 1)  # Check if all values are the same across price columns

    # Step 2: Check if Volume is NaN for those rows
    volume_empty_rows = price_same_rows & df['Volume'].isna()

    # Step 3: Replace NaN Volume with 0.0 (in float) for those rows
    df.loc[volume_empty_rows, 'Volume'] = 0.0
  

    print('Finaldataset',df.head(16))

    # Check for duplicate dates
    if 'Date' in df.columns:
        duplicate_dates = df['Date'].duplicated().sum()
        if duplicate_dates > 0:
            print(f"There are {duplicate_dates} duplicate date(s).")
            print("Duplicate rows:")
            print(df[df['Date'].duplicated(keep=False)])
        else:
            print("No duplicate dates found.")
    else:
        print("The column 'Date' does not exist in the DataFrame.")

    dfs[ticker]=df
 

Date           0
Open         548
High         550
Low          550
Close        550
Adj Close    544
Volume       561
dtype: int64
initialdataset           Date   Open          High           Low  Close     Adj Close  \
0   1980-03-17  290.0  3.100000e+02  2.900000e+02  290.0  2.900000e+02   
1   1980-03-18  290.0  2.900000e+02  2.900000e+02  290.0  2.900000e+02   
2   1980-03-19  290.0  3.100000e+02  2.900000e+02  290.0  2.900000e+02   
3   1980-03-20  290.0  2.900000e+02  2.900000e+02  290.0  2.900000e+02   
4   1980-03-21  290.0  2.900000e+02  2.900000e+02  290.0  2.900000e+02   
5   1980-03-24  290.0  2.900000e+02  2.700000e+02  270.0           NaN   
6   1980-03-25  270.0  2.700000e+02  2.700000e+02  270.0  2.700000e+02   
7   1980-03-26  270.0  2.700000e+02  2.700000e+02  270.0  2.700000e+02   
8   1980-03-27  270.0  2.700000e+02  2.700000e+02  270.0  2.700000e+02   
9   1980-03-28  270.0  2.700000e+02  2.700000e+02  270.0  2.700000e+02   
10  1980-03-31  270.0  2.800000e+02  2.